# INVASE Baseline, in 100-feature dataset

In [1]:
X_DIM = 100 # feature dimension Hyper-Param

In [2]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
# import argparse
from data_generation import generate_data
import os
import json
import pandas as pd
import time
import initpath_alg
#initpath_alg.init_sys_path()
import utilmlab
import data_loader_mlab


def array2str(a):
    s = ''
    for idx, el in enumerate(a):
        s += (' ' if idx > 0 else '') + '{:0.3f}'.format(el)
    return s


def one_hot_encoder(a):
    n_values = np.max(a) + 1
    return np.eye(n_values)[a]


def load_create_data(
        data_type,
        data_out,
        is_logging_enabled=True,
        fn_csv=None,
        label_nm=None):

    df_train, df_test, dset = None, None, None
    features = None
    if data_type in data_loader_mlab.get_available_datasets() + ['show'] \
       or fn_csv is not None:
        if fn_csv is not None:
            rval, dset = data_loader_mlab.load_dataset_from_csv(
                logger, fn_csv, label_nm)
        else:
            rval, dset = data_loader_mlab.get_dataset(data_type)
        assert rval == 0
        data_loader_mlab.dataset_log_properties(logger, dset)
        if is_logging_enabled:
            logger.info('warning no seed')
        df = dset['df']
        features = dset['features']
        labels = dset['targets']
        nsample = len(df)
        train_ratio = 0.8
        idx = np.random.permutation(nsample)
        ntrain = int(nsample * train_ratio)
        df_train = df.iloc[idx[:ntrain]]
        df_test = df.iloc[idx[ntrain:]]

        col_drop = utilmlab.col_with_nan(df)
        if is_logging_enabled and len(col_drop):
            print('warning: dropping features {}'
                  ', contains nan'.format(col_drop))
            time.sleep(2)

        features = [el for el in features if el not in col_drop]

        x_train = df_train[features].values
        y_train = df_train[labels].values
        x_test = df_test[features].values
        y_test = df_test[labels].values

        g_train, g_test = None, None

        y_train = one_hot_encoder(np.ravel(y_train))
        y_test = one_hot_encoder(np.ravel(y_test))
        if is_logging_enabled:
            logger.info('y: train:{} test:{}'.format(
                set(np.ravel(y_train)), set(np.ravel(y_test))))
    else:
        x_train, y_train, g_train = generate_data(
            n=train_N, data_type=data_type, seed=train_seed, out=data_out, x_dim = X_DIM)
        x_test,  y_test,  g_test = generate_data(
            n=test_N,  data_type=data_type, seed=test_seed,  out=data_out, x_dim = X_DIM)
    if is_logging_enabled:
        logger.info('{} {} {} {}'.format(
            x_train.shape,
            y_train.shape,
            x_test.shape,
            y_test.shape))
    return x_train, y_train, g_train, x_test, y_test, \
        g_test, df_train, df_test, dset, features




In [3]:
from IPython import embed
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(2)

# Generator (Actor) in PyTorch
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()

        self.l1 = nn.Linear(state_dim, 100)
        self.l2 = nn.Linear(100, 100)
        self.l3 = nn.Linear(100, action_dim)


    def forward(self, state):
        a = F.selu(self.l1(state))
        a = F.selu(self.l2(a))
        return torch.sigmoid(self.l3(a))
        
# Discriminator (Critic) in PyTorch    
# Critic in INVASE is a classifier that provide return signal
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(state_dim, 200)
        self.bn1 = nn.BatchNorm1d(num_features=200)
        self.l2 = nn.Linear(200, 200)
        self.bn2 = nn.BatchNorm1d(num_features=200)
        self.l3 = nn.Linear(200, 2)


    def forward(self, state, action):
        #sa = torch.cat([state, action], 1)
        sa = state * action

        q1 = F.selu(self.bn1(self.l1(sa)))
        q1 = F.selu(self.bn2(self.l2(q1)))
        q1 = self.l3(q1)

        return q1 # prob, actually the binary classification result with softmax activation (logits)
    
# Valuefunction (Baseline) in PyTorch   
# Valuefunction in INVASE is a classifier that provide return signal
class Baseline(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Baseline, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(state_dim, 200)
        self.bn1 = nn.BatchNorm1d(num_features=200)
        self.l2 = nn.Linear(200, 200)
        self.bn2 = nn.BatchNorm1d(num_features=200)
        self.l3 = nn.Linear(200, 2)


    def forward(self, state):
        #sa = torch.cat([state, action], 1)
        sa = state

        q1 = F.selu(self.bn1(self.l1(sa)))
        q1 = F.selu(self.bn2(self.l2(q1)))
        q1 = self.l3(q1)

        return q1 # prob, actually the binary classification result with softmax activation (logits)    

    
    
class PVS():
    # 1. Initialization
    '''
    x_train: training samples
    data_type: Syn1 to Syn 6
    '''
    def __init__(self, x_train, data_type, nepoch, is_logging_enabled=True, thres = 0.5):
        self.is_logging_enabled = is_logging_enabled
        self.latent_dim1 = 100      # Dimension of actor (generator) network
        self.latent_dim2 = 200      # Dimension of critic (discriminator) network
        
        self.batch_size = min(1000, x_train.shape[0])      # Batch size
        self.epochs = nepoch        # Epoch size (large epoch is needed due to the policy gradient framework)
        self.lamda = 99999.           # Hyper-parameter for the number of selected features 
        self.thres = thres
        '''lamda is number of selected features? is it the coefficient?'''
        
        
        self.input_shape = x_train.shape[1]     # Input dimension
        logger.info('input shape: {}'.format(self.input_shape))
        
        # Actionvation. (For Syn1 and 2, relu, others, selu)
        self.activation = 'relu' if data_type in ['Syn1','Syn2'] else 'selu'       
        
        
        self.generator = Actor(state_dim=self.input_shape, action_dim = self.input_shape).to(device)
        self.discriminator = Critic(state_dim=self.input_shape, action_dim = self.input_shape).to(device)
        self.valfunction = Baseline(state_dim=self.input_shape, action_dim = self.input_shape).to(device)
        
        
        self.generator_optimizer = torch.optim.Adam(self.generator.parameters(), lr=1e-4)#,weight_decay=1e-3)
        self.discriminator_optimizer = torch.optim.Adam(self.discriminator.parameters(), lr=1e-4)#,weight_decay=1e-3)
        self.valfunction_optimizer = torch.optim.Adam(self.valfunction.parameters(), lr=1e-4)#,weight_decay=1e-3)
        
    def my_loss(self, y_true, y_pred,lmd,thr):
        
        
        # dimension of the features
        d = y_pred.shape[1]        
        
        # Put all three in y_true 
        # 1. selected probability
        sel_prob = y_true[:,:d] # bs x d
        # 2. discriminator output
        dis_prob = y_true[:,d:(d+2)] # bs x 2
        # 3. valfunction output
        val_prob = y_true[:,(d+2):(d+4)] # bs x 2
        # 4. ground truth
        y_final = y_true[:,(d+4):] # bs x 2
        
        # A1. Compute the rewards of the actor network
        Reward1 = torch.sum( y_final * torch.log(dis_prob + 1e-8), axis = 1)  
        #reduce_sum(
        
        # A2. Compute the rewards of the actor network
        Reward2 = torch.sum( y_final * torch.log(val_prob + 1e-8), axis = 1)  

        # Difference is the rewards
        Reward = Reward1 - Reward2

        # B. Policy gradient loss computation. 
        #embed()
        loss1 = Reward * torch.sum( sel_prob * torch.log(y_pred + 1e-8) + (1-sel_prob) * torch.log(1-y_pred + 1e-8), axis = 1) - lmd *torch.mean( torch.abs(y_pred-thr), axis = 1)
        
        # C. Maximize the loss1
        loss = torch.mean(-loss1)
        #embed()
        return loss
    

    def Sample_M(self, gen_prob):
        # Shape of the selection probability
        n = gen_prob.shape[0]
        d = gen_prob.shape[1]
        # Sampling
        samples = np.random.binomial(1, gen_prob, (n,d))

        return samples

    #%% Training procedure
    def train(self, x_train, y_train, lmd,thr):

        # For each epoch (actually iterations)
        for epoch in range(self.epochs):

            #%% Train Discriminator
            # Select a random batch of samples
            idx = np.random.randint(0, x_train.shape[0], self.batch_size)
            x_batch = torch.as_tensor(x_train[idx,:]).float().to(device)
            y_batch = torch.as_tensor(np.argmax(y_train[idx,:],1)).long().to(device)
            
            # Generate a batch of probabilities of feature selection
            gen_prob = self.generator(x_batch).cpu().detach().numpy()
            #print(gen_prob)
            # Sampling the features based on the generated probability
            sel_prob = self.Sample_M(gen_prob)
            '''sel_prob is the mask'''
            
            # Compute the prediction of the critic based on the sampled features (used for generator training)
            dis_prob = self.discriminator(x_batch, torch.as_tensor(sel_prob).float().to(device))
            
            # Train the discriminator
            #d_loss = self.discriminator.train_on_batch([x_batch, sel_prob], y_batch)
            loss_func_c = nn.CrossEntropyLoss()
            self.discriminator_optimizer.zero_grad()
            critic_loss = loss_func_c(dis_prob, y_batch)
            critic_loss.backward()
            self.discriminator_optimizer.step()

            #%% Train Valud function

            # Compute the prediction of the baseline based on the sampled features (used for generator training)
            val_prob = self.valfunction(x_batch)#.cpu().detach().numpy()
            
            # Train the baseline model
            #v_loss = self.valfunction.train_on_batch(x_batch, y_batch)
            loss_func_v = nn.CrossEntropyLoss()
            self.valfunction_optimizer.zero_grad()
            value_loss = loss_func_v(val_prob, y_batch)
            value_loss.backward()
            self.valfunction_optimizer.step()
            
            
            #%% Train Generator
            # Use three things as the y_true: sel_prob, dis_prob, and ground truth (y_batch)
            '''
            sel_prob: the mask generated by bernulli sampler [bs, d]
            dis_prob: prediction of the critic               [bs, 2]
            val_prob: prediction of the baseline model       [bs, 2]
            y_batch: batch of y_train                        [bs, 2]
            all of those variables are 'placeholders'
            '''
            
            y_batch_final = torch.as_tensor(np.concatenate( (sel_prob, np.asarray(torch.softmax(torch.as_tensor(dis_prob),-1).cpu().detach().numpy()), np.asarray(torch.softmax(torch.as_tensor(val_prob),-1).cpu().detach().numpy()), y_train[idx,:]), axis = 1 )).to(device)
            # Train the generator
            
            actor_pred = self.generator(x_batch)
            self.generator_optimizer.zero_grad()
            actor_loss = self.my_loss(y_batch_final,actor_pred,lmd,thr)
            actor_loss.backward()
            self.generator_optimizer.step()
            
            #%% Plot the progress
            dialog = 'Epoch: ' + '{:6d}'.format(epoch) + ', d_loss (Acc)): '
            dialog += '{:0.3f}'.format(critic_loss) + ', v_loss (Acc): '
            dialog += '{:0.3f}'.format(value_loss) + ', g_loss: ' + '{:+6.4f}'.format(actor_loss)

            if epoch % 100 == 0:
                logger.info('{}'.format(dialog))
    
    #%% Selected Features        
    def output(self, x_train):
        
        gen_prob = self.generator(x_train).cpu().detach().numpy()
        
        return np.asarray(gen_prob)
     
    #%% Prediction Results 
    def get_prediction(self, x_train, m_train):
        
        val_prediction = self.valfunction(x_train).cpu().detach().numpy()
        
        dis_prediction = self.discriminator(x_train, m_train).cpu().detach().numpy()
        
        return np.asarray(val_prediction), np.asarray(dis_prediction)

In [5]:
REAL_LMD = 1.0 # 0.0 - 0.5


import time
elapsed_time = []

class init_arg(object):
    def __init__(self, it = 10000, o = 'feature_score.csv.gz', dataset = None, i= None, target = None):
        self.it = it
        self.o = o
        self.dataset = dataset
        self.i = i
        self.target = target
        
        
progress_loger = []

for DATASET in ['Syn4','Syn5','Syn6']:
    sub_logger = []
    args = init_arg(dataset = DATASET, it = 10000, )
    ocsv = args.o # 'feature_score.csv.gz'
    odir = os.path.dirname(ocsv)
    odir = '.' if not len(odir) else odir
    fn_csv = args.i #'data.csv'
    label_nm = args.target # 'target'
    nepoch = args.it
    logger = utilmlab.init_logger(odir)

    dataset = args.dataset

    assert dataset is not None or fn_csv is not None
    assert fn_csv is None or label_nm is not None

    # Data output can be either binary (Y) or Probability (Prob)
    data_out_sets = ['Y', 'Prob']
    data_out = data_out_sets[0]

    logger.info('invase: {} {} {} {}'.format(dataset, nepoch, odir, data_out))

    # Number of Training and Testing samples
    train_N = 10000
    test_N = 10000

    # Seeds (different seeds for training and testing)
    train_seed = 0
    test_seed = 1

    x_train, y_train, _, x_test, y_test, g_test, df_train, df_test, \
        dset, features = load_create_data(
            dataset,
            data_out,
            is_logging_enabled=True,
            fn_csv=fn_csv,
            label_nm=label_nm)
    

    start_time = time.time()
    for thres_i in [0.0]:
        Predict_Out_temp = np.zeros([3, 2])    

        PVS_Alg = PVS(x_train, dataset, 100, thres=thres_i)

        for train_epoch in range(int(nepoch/100)):
            PVS_Alg.generator.train()
            Lmd = 1.0 #train_epoch*100/nepoch * REAL_LMD
            Thr = 0.0 #0.5*(1 - train_epoch*100/nepoch)
            print('now at training epoch number', int(train_epoch * 100),'hyp-params: lamda %.4f prior %.4f'%(Lmd,Thr))
            PVS_Alg.train(x_train, y_train, lmd = Lmd , thr = Thr)
            # 3. Get the selection probability on the testing set
            #Sel_Prob_Test = PVS_Alg.output(x_test)



            '''recurssive generation'''
            input_batch = x_test * 1.0
            sel_prob_tot = 1.0
            for recur_time in range(4):
                print('rec time now',recur_time,'dataset now:',DATASET)
                PVS_Alg.generator.eval()
                gen_prob = PVS_Alg.generator(torch.as_tensor(input_batch).float().to(device))
                #sel_prob = PVS_Alg.Sample_M(gen_prob)
                sel_prob = 1.*(gen_prob > 0.5)
                sel_prob_tot_0 = sel_prob_tot * 1.0
                sel_prob_tot = torch.as_tensor(sel_prob * sel_prob_tot).to(device)
                input_batch = sel_prob_tot * torch.as_tensor(input_batch).to(device)

                score = sel_prob_tot
                #print('score',score)



                # 4. Selected features
                # 5. Prediction
                val_predict, dis_predict = PVS_Alg.get_prediction(torch.as_tensor(x_test).float().to(device), score)
                val_predict = torch.softmax(torch.as_tensor(val_predict),-1).cpu().numpy()
                dis_predict = torch.softmax(torch.as_tensor(dis_predict),-1).cpu().numpy()


                def performance_metric(score, g_truth):

                    n = len(score)
                    Temp_TPR = np.zeros([n,])
                    Temp_FDR = np.zeros([n,])

                    for i in range(n):

                        # TPR    
                        # embed()
                        TPR_Nom = np.sum((score[i,:].cpu().detach().numpy() * g_truth[i,:]))
                        TPR_Den = np.sum(g_truth[i,:])
                        Temp_TPR[i] = 100 * float(TPR_Nom)/float(TPR_Den+1e-8)

                        # FDR
                        FDR_Nom = np.sum((score[i,:].cpu().detach().numpy() * (1-g_truth[i,:])))
                        FDR_Den = np.sum(score[i,:].cpu().detach().numpy())
                        Temp_FDR[i] = 100 * float(FDR_Nom)/float(FDR_Den+1e-8)

                    return np.mean(Temp_TPR), np.mean(Temp_FDR),\
                        np.std(Temp_TPR), np.std(Temp_FDR)

                #%% Output

                TPR_mean, TPR_std = -1, 0
                FDR_mean, FDR_std = -1, 0
                if g_test is not None:
                    TPR_mean, FDR_mean, TPR_std, FDR_std = performance_metric(
                        score, g_test)

                    logger.info('TPR mean: {:0.1f}%  std: {:0.1f}%'.format(
                        TPR_mean, TPR_std))
                    logger.info('FDR mean: {:0.1f}%  std: {:0.1f}%'.format(
                        FDR_mean, FDR_std))
                else:
                    logger.info('no ground truth relevance')



                #%% Performance Metrics
                Predict_Out_temp[0,0] = roc_auc_score(y_test[:,1], val_predict[:,1])
                Predict_Out_temp[1,0] = average_precision_score(y_test[:,1], val_predict[:,1])
                Predict_Out_temp[2,0] = accuracy_score(y_test[:,1], 1. * (val_predict[:,1]>0.5) )

                Predict_Out_temp[0,1] = roc_auc_score(y_test[:,1], dis_predict[:,1])
                Predict_Out_temp[1,1] = average_precision_score(y_test[:,1], dis_predict[:,1])
                Predict_Out_temp[2,1] = accuracy_score(y_test[:,1], 1. * (dis_predict[:,1]>0.5) )
                print(Predict_Out_temp)
                sub_logger.append((Predict_Out_temp,TPR_mean,FDR_mean,TPR_std,FDR_std))
        progress_loger.append(sub_logger)
    elapsed_time.append(time.time() - start_time)
    print('PyTorch Version: elapsed time for {}: 100 feature, 10000 sample:'.format(DATASET),np.round(elapsed_time,4),'sec.')


invase: Syn4 10000 . Y
(10000, 100) (10000, 2) (10000, 100) (10000, 2)
input shape: 100
Epoch:      0, d_loss (Acc)): 0.762, v_loss (Acc): 0.765, g_loss: +0.7014


now at training epoch number 0 hyp-params: lamda 1.0000 prior 0.0000
rec time now 0 dataset now: Syn4


TPR mean: 38.7%  std: 23.4%
FDR mean: 96.4%  std: 2.3%


[[0.50776901 0.51515415]
 [0.53512117 0.53508962]
 [0.5135     0.522     ]]
rec time now 1 dataset now: Syn4


TPR mean: 26.2%  std: 20.3%
FDR mean: 96.5%  std: 2.9%


...

[[0.65602031 0.86131181]
 [0.67108941 0.86969561]
 [0.6158     0.7846    ]]
rec time now 2 dataset now: Syn6


TPR mean: 90.1%  std: 10.0%
FDR mean: 43.7%  std: 6.2%


[[0.65602031 0.86131181]
 [0.67108941 0.86969561]
 [0.6158     0.7846    ]]
rec time now 3 dataset now: Syn6


TPR mean: 90.1%  std: 10.0%
FDR mean: 43.7%  std: 6.2%


[[0.65602031 0.86131181]
 [0.67108941 0.86969561]
 [0.6158     0.7846    ]]
PyTorch Version: elapsed time for Syn6: 100 feature, 10000 sample: [1046.6599 1077.6392 1053.6789] sec.
